# Initialization

In [1]:
%load_ext autoreload
%autoreload 2

# Use only one GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../isanlp/src/')
sys.path.append('../../src/isanlp_srl_framebank/')
sys.path.append('../../libs/')
sys.path.append('../../libs/pylingtools/')

In [2]:

# Supress tensorflow memory appetites

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.log_device_placement=True
sess = tf.Session(config=config)

from tensorflow.python.keras import backend as K
K.set_session(sess)

In [3]:
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
W0517 06:32:42.619653 139967747114752 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [4]:
# Check available GPUs

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [5]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12838416873275428403, name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2095912996844436686
 physical_device_desc: "device: XLA_GPU device", name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2305558900680292752
 physical_device_desc: "device: XLA_CPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10791449396
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11958355179376094636
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:0c:00.0, compute capability: 6.1"]

In [6]:
import os
import time
import isanlp
import json
import pickle

import numpy as np
np.random.seed(31)

import pandas as pd

In [67]:
from tqdm import tqdm_notebook as tqdm

# Data loading

In [7]:
cleared_corpus_path = '../../data/cleared_corpus.json'

In [8]:
with open(cleared_corpus_path, 'r') as f:
    examples = json.load(f)

In [9]:
ling_data_path = '../../data/results_final_fixed.pckl'
with open(ling_data_path, 'rb') as f:
    ling_data = pickle.load(f)

ling_data_cache = {k: v for k,v in ling_data}

In [65]:
error_examples = {}

# Feature construction

In [68]:
from isanlp.annotation_repr import CSentence
from convert_corpus_to_brat import make_text


def find_address_by_offset(offset, ling_ann):
    for tok_num, tok in enumerate(ling_ann['tokens']):
        if tok.begin <= offset and offset < tok.end:
            break
    
    for sent_num, sent in enumerate(ling_ann['sentences']):
        if sent.begin <= tok_num and tok_num < sent.end:
            break
    
    return sent_num, tok_num - sent.begin


def process_arg_pred(feature_extractor, ling_cache, ex_id, pred, args, example):
    feature_sets = list()
    
    text, offset_index = make_text(example, 0)
    ling_ann = ling_cache[ex_id]
    
    pred_offset = offset_index[(pred[0], pred[1])]
    pred_ling_sent, pred_ling_word = find_address_by_offset(pred_offset, ling_ann)
    
    for arg in args:
        arg_offset = offset_index[(arg[0], arg[1])]
        arg_ling_sent, arg_ling_word = find_address_by_offset(arg_offset, ling_ann)
        
        fb_pred_word = example[pred[0]][pred[1]]
        fb_arg_word = example[arg[0]][arg[1]]
        
        if arg_ling_sent != pred_ling_sent:
            error_examples[ex_id] = {
                'reason': 'sent_mismatch',
                'arg': arg_ling_sent,
                'pred': pred_ling_sent
            }
            continue
            
        sentence = ling_ann['sentences'][pred_ling_sent]
        tokens = [tok.text for tok in ling_ann['tokens']]
        tokens = tokens[sentence.begin:sentence.end]
        
        role = fb_arg_word['rolepred1']

        features = feature_extractor.extract_features(pred_ling_word, 
                                                      arg_ling_word, 
                                                      ling_ann['postag'][arg_ling_sent],
                                                      ling_ann['morph'][arg_ling_sent],
                                                      ling_ann['lemma'][arg_ling_sent],
                                                      ling_ann['syntax_dep_tree'][arg_ling_sent])

                    
        feature_sets.append((features, role, ex_id, tokens, arg_ling_word, pred_ling_word))
    
    return feature_sets


def process_example(feature_extractor, ling_cache, ex_id, sentences):
    pred = None
    args = list()
    for sent_num, sent in enumerate(sentences):
        for word_num, word in enumerate(sent):
            if 'rank' in word and word['rank'] == 'Предикат':
                pred = (sent_num, word_num)
            elif 'rolepred1' in word:
                args.append((sent_num, word_num))
    
    return process_arg_pred(feature_extractor, ling_cache, ex_id, pred, args, sentences)


num_of_errors = 0
def prepare_train_data(examples, ling_data_cache, feature_extractor):
    feature_sets = []
    for ex_num, (ex_id, ex) in tqdm(list(enumerate(examples))):    
        #if ex_num % 100 == 0:
        #    print('{0:.2f}%'.format((ex_num / len(examples)) * 100.))
            
        feature_sets += process_example(feature_extractor, ling_data_cache, ex_id, ex)

    print('Number of training examples:', len(feature_sets))
    return feature_sets

In [69]:
main_model_path_root = '../../data/models_new/'

In [70]:
#!!!: Choose feature model here
from isanlp_srl_framebank.processor_srl_framebank import FeatureModelDefault
feature_model = FeatureModelDefault()
main_model_path = os.path.join(main_model_path_root, 'known_preds')

# from isanlp_srl_framebank.processor_srl_framebank import FeatureModelUnknownPredicates
# feature_model = FeatureModelUnknownPredicates()
# main_model_path = os.path.join(main_model_path_root, 'unknown_preds')

#with open(os.path.join(main_model_path, 'feature_model.pckl'), 'wb') as f:
#    pickle.dump(feature_model, f)

In [71]:
(vars(ling_data_cache['30587']['sentences'][0]))

{'begin': 0, 'end': 17}

In [72]:
len(error_examples)

0

In [73]:
feature_sets = prepare_train_data(examples, ling_data_cache, feature_model)

data_for_pandas = []
for example in feature_sets:
    data_for_pandas_ex = {}
    data_for_pandas_ex['role'] = example[1]
    data_for_pandas_ex['ex_id'] = example[2]
    data_for_pandas_ex['tokens'] = example[3]
    data_for_pandas_ex['arg_address'] = example[4]
    data_for_pandas_ex['prd_address'] = example[5]
    for elem in example[0]:
        for subelem in elem:
            if subelem is not None:
                data_for_pandas_ex.update(subelem)
    
    data_for_pandas.append(data_for_pandas_ex)
    
pd_data = pd.DataFrame(data_for_pandas)
pd_data = pd_data.sample(frac=1)
pd_data[:10]
del data_for_pandas


Number of training examples: 57552


In [76]:
N_verify = 10
for i in np.random.choice(len(pd_data), size=N_verify):
    print("-"*60)
    obj = pd_data.iloc[i]
    print(f"Argument position: {obj.arg_address}")
    print(f"Argument lemma: {obj.arg_lemma}")
    print("--")
    print(f"Predicat position: {obj.prd_address}")
    print(f"Predicat lemma: {obj.pred_lemma}")
    print("--")
    print(f"Distance {int(obj.dist)}")
    print("--")
    print(f"Sentence tokens: {list(enumerate(obj.tokens))}")

------------------------------------------------------------
Argument position: 0
Argument lemma: я_PRON
--
Predicat position: 3
Predicat lemma: заводить_VERB
--
Distance 3
--
Sentence tokens: [(0, 'Я'), (1, 'всё'), (2, 'собирался'), (3, 'завести'), (4, 'сберкнижку'), (5, ','), (6, 'чтобы'), (7, 'продолжить'), (8, 'прерванное'), (9, 'своё'), (10, 'высшее'), (11, 'образование'), (12, ','), (13, 'и'), (14, 'эти'), (15, 'благие'), (16, 'порывы'), (17, 'тревожили'), (18, 'меня'), (19, 'до'), (20, 'тех'), (21, 'пор'), (22, ','), (23, 'пока'), (24, 'я'), (25, 'не'), (26, 'обнаружил'), (27, 'в'), (28, 'себе'), (29, 'склонности'), (30, 'к'), (31, 'писательству'), (32, '.')]
------------------------------------------------------------
Argument position: 1
Argument lemma: человек_NOUN
--
Predicat position: 8
Predicat lemma: греть_VERB
--
Distance 7
--
Sentence tokens: [(0, 'Последние'), (1, 'люди'), (2, 'будут'), (3, 'выползать'), (4, 'откуда'), (5, '-'), (6, 'то'), (7, 'и'), (8, 'греть'), (9, '

# Убрать все фичи предикатного слова для модели Unknown Predicates

# Preprocessing

In [77]:
y_stat = pd_data.loc[:, 'role'].value_counts()
drop_ys = y_stat[y_stat < 180].index
clear_data = pd_data.drop(pd_data[pd_data.loc[:, 'role'].isin(drop_ys)].index)

In [78]:
repl_roles = {
    'агенс - субъект восприятия' : 'субъект восприятия',
    'агенс - субъект ментального состояния' : 'субъект ментального состояния',
    'результат / цель' : 'результат',
    'место - пациенс' : 'место',
    'говорящий - субъект психологического состояния' : 'субъект психологического состояния'
}


def normalize_single_region(data, rep, val):
    data.loc[:, 'role'] = data.loc[:, 'role'].str.replace(rep, val)


for rep, val in repl_roles.items():
    normalize_single_region(clear_data, rep, val)
    
number_of_roles = len(clear_data.loc[:, 'role'].value_counts().index)
print('Number of roles: ', number_of_roles)
clear_data.loc[:, 'role'].value_counts()

Number of roles:  44


агенс                                 6147
пациенс                               5362
тема                                  3656
субъект психологического состояния    3250
субъект перемещения                   3011
причина                               2502
говорящий                             2365
место                                 2185
содержание действия                   1874
содержание мысли                      1817
содержание высказывания               1792
конечная точка                        1772
результат                             1452
пациенс перемещения                   1356
стимул                                1271
субъект ментального состояния         1223
адресат                                941
субъект восприятия                     901
контрагент                             831
эффектор                               739
субъект социального отношения          598
начальная точка                        588
предмет высказывания                   548
способ     

In [79]:
list(clear_data.loc[:, 'role'].drop_duplicates().values)

['пациенс',
 'содержание действия',
 'содержание мысли',
 'конечная точка',
 'субъект психологического состояния',
 'место',
 'конечный посессор',
 'содержание высказывания',
 'контрагент',
 'сфера',
 'потенциальная угроза',
 'тема',
 'субъект поведения',
 'каузатор',
 'агенс',
 'ситуация в фокусе',
 'причина',
 'говорящий',
 'эталон',
 'пациенс перемещения',
 'стимул',
 'эффектор',
 'субъект перемещения',
 'субъект ментального состояния',
 'начальная точка',
 'субъект восприятия',
 'статус',
 'результат',
 'способ',
 'пациенс социального отношения',
 'контрагент социального отношения',
 'субъект социального отношения',
 'признак',
 'источник звука',
 'адресат',
 'цель',
 'срок',
 'исходный посессор',
 'предмет мысли',
 'субъект физиологической реакции',
 'предмет высказывания',
 'потенциальный пациенс',
 'признак действия',
 'траектория']

In [80]:
y_orig = clear_data.loc[:, 'role']
X_orig = clear_data.drop('role', axis = 1)
X_orig.shape

(52751, 20)

In [81]:
from sklearn.preprocessing import LabelBinarizer
import pickle

label_encoder = LabelBinarizer()
y = label_encoder.fit_transform(y_orig)

with open(main_model_path + '/label_encoder.pckl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [82]:
y.shape

(52751, 44)

## Embedding

In [104]:
from gensim.models import KeyedVectors

embeddings_path = '../../data/ruscorpora_upos_skipgram_300_5_2018.vec'
embeddings = KeyedVectors.load_word2vec_format(embeddings_path, binary=False)
print('Embedding size: ', embeddings.vector_size)

Embedding size:  300


In [105]:
import multiprocessing as mp


def make_embeded_form(word):
    if word:
        #return word[1].encode('utf8')
        return u"{}_{}".format(word[1], word[0])
    else:
        return word


class Embedder_map:
    def __init__(self, embeddings, X):
        self.X_ = X
        self.embeddings_ = embeddings

    def __call__(self, i):  
        result = np.zeros((len(self.X_[0]), 
                           self.embeddings_.vector_size))

        for j in range(len(self.X_[0])):
            word = self.X_[i][j]
            tag = word[0] if word else str()
            
            if tag == ARG_SPECIAL_TAG or tag == ARG_SPECIAL_TAG:
                result[j, :] = np.ones(self.embeddings_.vector_size)
            elif word and word in embeddings:
                result[j, :] = self.embeddings_[word]

        return result


def embed(X):
    pool = mp.Pool(4)
    result = pool.map(Embedder_map(embeddings, X), X.index, 1000)
    pool.close()
    return np.asarray(result)


In [106]:
X_orig.columns

Index(['Animacy_arg', 'Aspect_arg', 'Gender_arg', 'Number_arg', 'Tense_arg',
       'Valency_arg', 'VerbForm_arg', 'arg_address', 'arg_case', 'arg_lemma',
       'arg_pos', 'dist', 'ex_id', 'prd_address', 'pred_lemma', 'pred_pos',
       'prepos', 'rel_pos', 'syn_link_name'],
      dtype='object')

In [22]:
%%time

arg_context_embedded = embed(X_orig.loc[:, 'arg_context_lemmas'])

KeyError: 'the label [arg_context_lemmas] is not in the [columns]'

In [ ]:
%%time

pred_context_embedded = embed(X_orig.loc[:, 'pred_context_lemmas'])

In [108]:
class Embedder_single_map:
    def __init__(self, embeddings, X):
        self.X_ = X
        self.embeddings_ = embeddings

    def __call__(self, i):
        #word = make_embeded_form(self.X_[i])
        word = self.X_[i]
        if word in self.embeddings_:
            return self.embeddings_[word]
        else:
            return np.zeros((self.embeddings_.vector_size,))

        
def embed_single(embeddings, X):
    pool = mp.Pool(4)
    result = pool.map(Embedder_single_map(embeddings, X), X.index, 1000)
    pool.close()
        
    return np.asarray(result)

In [83]:
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz", elmo_output_names=['elmo'])

Using TensorFlow backend.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0517 08:16:47.331135 139967747114752 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [84]:
def elmo_embed(embeddings, tokens, word_idx1, word_idx2):
    embedded = embeddings([tokens])[0]
    return embedded[min(word_idx1, len(tokens)-1)], embedded[min(word_idx2, len(tokens)-1)] 

In [85]:
%%time
embedded_verbs = []
embedded_args  = []
for i in tqdm(range(len(X_orig))):
    try:
        if i % 100 == 0:
            with open("./log.txt", 'a', encoding='utf-8') as log:
                print(f"Processed {i} examples", file=log)
        obj = X_orig.iloc[i]
        verb_idx = obj.prd_address
        arg_idx = obj.arg_address
        tokens = obj.tokens
        embed_verb, embed_arg = elmo_embed(elmo, tokens, verb_idx, arg_idx)
        embedded_verbs.append(embed_verb)
        embedded_args.append(embed_arg)
    except Exception as e:
        with open("./log.txt", 'a', encoding='utf-8') as log:
            print(f"Error while processing example {i}={X_orig.iloc[i]}: {e}", file=log)


CPU times: user 2h 40min 23s, sys: 15min 36s, total: 2h 56min
Wall time: 1h 20min 57s


In [86]:
e_verbs = np.stack(embedded_verbs)
e_args  = np.stack(embedded_args)

print(e_verbs.shape, e_args.shape)

(52751, 1024) (52751, 1024)


In [87]:
np.save("../../data/elmo_verbs.npy", e_verbs)
np.save("../../data/elmo_args.npy", e_args)

In [112]:
%%time

embedded_verbs = embed_single(embeddings, pd.Series(list(zip(X_orig.pred_pos, X_orig.pred_lemma)), 
                                         index = X_orig.index))

print(embedded_verbs.shape)
print((np.linalg.norm(embedded_verbs, axis = 1) < 0.001).sum())
print(clear_data[(np.linalg.norm(embedded_verbs, axis = 1) < 0.001)].pred_lemma.value_counts().shape)

OSError: [Errno 12] Cannot allocate memory

In [63]:
%%time

# embedded_args = embed_single(pd.Series(list(zip(X_orig.arg_pos, X_orig.arg_lemma)), 
#                                        index = X_orig.index))
embedded_args = embed_single(embeddings, X_orig.arg_lemma)
print(embedded_args.shape)
print((np.linalg.norm(embedded_args, axis = 1) < 0.001).sum())

(30371, 300)
10342
CPU times: user 22 s, sys: 14.3 s, total: 36.3 s
Wall time: 42.3 s


## Vectorizing categorial features

In [89]:
X_orig = X_orig.drop(['tokens'], axis=1)

In [90]:
from sklearn.feature_extraction import DictVectorizer

#morph_feats = ['pos', 'case', 'anim', 'vform', 'zform', 'shform', 'pform', 'vvform', 'nform', 'time']

# all_feats = (['pred_lemma', 'rel_pos'] + 
#              ['arg_' + e for e in morph_feats] + 
#              ['pred_' + e for e in morph_feats])

# all_feats = (['pred_lemma', 'rel_pos', 'arg_prep'] + 
#              ['arg_' + e for e in morph_feats] + 
#              ['pred_' + e for e in morph_feats])

# all_feats = (['pred_lemma', 'rel_pos', 'arg_prep', 'link_name'] + 
#              ['arg_' + e for e in morph_feats] + 
#              ['pred_' + e for e in morph_feats])

#all_feats = ['pred_lemma', 'rel_pos', 'pred_pos', 'arg_case', 'syn_link_name', 'arg_pos', 'prepos', 'dist']

#categ_feats = [e for e in all_feats if X_orig[e].dtype in [str, object]]
#not_categ = [e for e in all_feats if e not in categ_feats]

#pred_lemma_vectorizer.fit_transform(X_orig.loc[:, ['pred_lemma']].to_dict(orient = 'records'))

not_categ_features = {'arg_address', 'ex_id', 'rel_pos', 'arg_lemma'}
categ_feats = [name for name in X_orig.columns if name not in not_categ_features] 
not_categ = ['rel_pos']
print('Category features:\n', categ_feats)
print('Not category features:\n', not_categ)

vectorizer = DictVectorizer(sparse = False)
one_hot_feats = vectorizer.fit_transform(X_orig.loc[:, categ_feats].to_dict(orient = 'records'))
print(one_hot_feats.shape)

with open(main_model_path + '/feature_encoder.pckl', 'wb') as f:
    pickle.dump(vectorizer, f)

Category features:
 ['Animacy_arg', 'Aspect_arg', 'Gender_arg', 'Number_arg', 'Tense_arg', 'Valency_arg', 'VerbForm_arg', 'arg_case', 'arg_pos', 'dist', 'prd_address', 'pred_lemma', 'pred_pos', 'prepos', 'syn_link_name']
Not category features:
 ['rel_pos']
(52751, 931)


In [101]:
np.save("../../data/labnpnpels.npy", y)

In [102]:
np.save("../../data/plain_features.npy", plain_features)

In [91]:
not_categ_columns = np.concatenate(tuple(X_orig.loc[:, e].as_matrix().reshape(-1, 1) for e in not_categ), axis =1)
plain_features = np.concatenate((one_hot_feats, not_categ_columns), axis = 1)
plain_features.shape

/root/.pyenv/versions/3.6.7/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(52751, 932)

In [92]:
del not_categ_columns
del one_hot_feats

# Model construction

In [93]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Convolution1D, Dropout, MaxPooling1D
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import TimeDistributed
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.layers import RepeatVector
from tensorflow.python.keras.layers import Permute
from tensorflow.python.keras.layers import Lambda
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import Concatenate
from tensorflow.python.keras.layers import Bidirectional
from tensorflow.python.keras.layers import Masking
from gensim.models import Word2Vec

In [94]:
def construct_plain_model(input_shape):
    print('Plain model.')
    
    plain_model = Sequential()
    plain_model.add(Dense(600, 
                          #input_shape=(plain_features.shape[1],), 
                          input_shape = input_shape,
                          activation = 'relu'))
    plain_model.add(Dropout(0.3))
    
    plain_model.add(Dense(400))
    plain_model.add(BatchNormalization())
    plain_model.add(Activation('relu'))
    plain_model.add(Dropout(0.3))
    
    plain_model.add(Dense(number_of_roles))
    plain_model.add(BatchNormalization())
    plain_model.add(Activation('softmax'))
    
    plain_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return plain_model

In [95]:
def construct_plain_model_sparse(categ_size, emb_size, number_of_roles):    
    input_plain = Input(shape=(categ_size,), name = 'input_categorical')
    input_pred_embed = Input(shape=(emb_size,), name = 'pred_embed')
    input_arg_embed = Input(shape=(emb_size,), name = 'arg_embed')
    
    plain = Dense(400)(input_plain)
    plain = BatchNormalization()(plain)
    plain = Activation('relu')(plain)
    
    def embed_submodel(inpt):
        embed = Dense(100)(inpt)
        embed = BatchNormalization()(embed)
        embed = Activation('relu')(embed)
        return embed
    
    embed_pred = embed_submodel(input_pred_embed)
    embed_arg = embed_submodel(input_arg_embed)
    
    final = Concatenate(axis = 1)([embed_pred, embed_arg, plain])
    final = Dropout(0.3)(final)
    final = Dense(400)(final)
    final = BatchNormalization()(final)
    final = Activation('relu')(final)
    final = Dropout(0.3)(final)
    final = Dense(number_of_roles)(final)
    final = BatchNormalization()(final)
    final = Activation('softmax')(final)
    
    model = Model([input_arg_embed, input_pred_embed, input_plain], final)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Experiments

## Experiments with in-domain test

In [35]:
# Use only for experiments with in-domain test. Do not use for training

### Plain model

In [99]:
model = construct_plain_model((plain_features.shape[1],))
print(model.summary())
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit(plain_features, y, epochs=15, batch_size=300, validation_split = 0.1, 
          shuffle=True, callbacks = [early_stopping])

Plain model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 600)               559800    
_________________________________________________________________
dropout_6 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 400)               240400    
_________________________________________________________________
batch_normalization_6 (Batch (None, 400)               1600      
_________________________________________________________________
activation_6 (Activation)    (None, 400)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 44)                17644   

### Sparse

In [100]:
model = construct_plain_model_sparse(plain_features.shape[1], 1024, y.shape[1])
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit([e_args, e_verbs, plain_features], y, epochs=16, batch_size=300, 
          validation_split = 0.1, shuffle=True, callbacks = [early_stopping])
model.save(os.path.join(main_model_path, 'neural_model.h5'))

Train on 47475 samples, validate on 5276 samples
Epoch 1/16
47475/47475 [==============================] - 4s 79us/step - loss: 2.3996 - acc: 0.4625 - val_loss: 1.8635 - val_acc: 0.6522
Epoch 2/16
47475/47475 [==============================] - 2s 50us/step - loss: 1.3493 - acc: 0.7272 - val_loss: 1.3605 - val_acc: 0.7595
Epoch 3/16
47475/47475 [==============================] - 2s 49us/step - loss: 0.9969 - acc: 0.7882 - val_loss: 0.9988 - val_acc: 0.7964
Epoch 4/16
47475/47475 [==============================] - 2s 50us/step - loss: 0.8276 - acc: 0.8152 - val_loss: 0.8172 - val_acc: 0.8099
Epoch 5/16
47475/47475 [==============================] - 2s 50us/step - loss: 0.7071 - acc: 0.8358 - val_loss: 0.7297 - val_acc: 0.8228
Epoch 6/16
47475/47475 [==============================] - 2s 50us/step - loss: 0.6246 - acc: 0.8503 - val_loss: 0.6848 - val_acc: 0.8251
Epoch 7/16
47475/47475 [==============================] - 2s 50us/step - loss: 0.5552 - acc: 0.8619 - val_loss: 0.6295 - val_acc:

In [78]:
from tensorflow.python.keras.backend import clear_session
clear_session()

### Context

In [79]:
%%time

model = construct_graph_lstm_model((plain_features.shape[1],))
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit([arg_context_embedded, pred_context_embedded, embedded_args, embedded_verbs, plain_features], y, 
          epochs=15, batch_size=64, validation_split = 0.1, 
          shuffle=True, callbacks = [early_stopping])

Context model.


NameError: name 'pred_context_embedded' is not defined

## Experiements with out-of-domain test

In [ ]:
# Use only for out of domain experiments. Do not use for training

In [80]:
def evaluate_out_of_domain(model, X_train, y_train, X_test, y_test):
    final_res = list()
    N_ITERATIONS = 5
    for i in xrange(N_ITERATIONS):
        print('Eval iter:', i + 1, '/', N_ITERATIONS)
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, 
                                       patience=2, verbose=0, mode='auto')
        model.fit(X_train, y_train, nb_epoch=15, 
                  batch_size=64, validation_split = 0.1, 
                  shuffle=True, callbacks = [early_stopping],
                 verbose = 0)

        ev_res = evaluate_model(model, X_test, y_test)
        print()
        print(pd.DataFrame([ev_res], columns = ['keras_accur', 'keras_loss', 'f1_micro', 'f1_macro', 'accur']))
        final_res.append(ev_res)
    
    return np.array(final_res)

In [81]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def evaluate_model(model, X_test, y_test):
    keras_eval = model.evaluate(X_test, y_test)
    pred = model.predict(X_test).argmax(axis = 1)
    f1_micro = f1_score(pred, y_test.argmax(axis = 1), average = 'micro')
    f1_macro = f1_score(pred, y_test.argmax(axis = 1), average = 'macro')
    accur = accuracy_score(pred, y_test.argmax(axis = 1))
    
    return np.array(list(keras_eval) + [f1_micro, f1_macro, accur])

### Simple

In [82]:
model = construct_plain_model((ind_plain_features.shape[1],))
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit(ind_plain_features, ind_y, nb_epoch=15, batch_size=64, validation_split = 0.1, 
          shuffle=True, callbacks = [early_stopping])

#model.evaluate(ood_plain_features, ood_y)
ev_res = evaluate_model(model, [ood_plain_features], ood_y)
print()
print(pd.DataFrame([ev_res], columns = ['keras_accur', 'keras_loss', 'f1_micro', 'f1_macro', 'accur']))

NameError: name 'ind_plain_features' is not defined

In [50]:
model = construct_plain_model((ind_plain_features.shape[1],))
model_eval = evaluate_out_of_domain(model, ind_plain_features, ind_y, ood_plain_features, ood_y)
print(model_eval)
describe_cv_result(model_eval)

NameError: name 'ind_plain_features' is not defined

### Complex

In [51]:
model = construct_plain_model_sparse((ind_plain_features.shape[1],))
model.summary()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit([ind_arg_embed, ind_pred_embed, ind_plain_features], ind_y, nb_epoch=20, batch_size=64, validation_split = 0.1, 
          shuffle=True, callbacks = [early_stopping])
#model.evaluate([ood_arg_embed, ood_pred_embed, ood_plain_features], ood_y)

ev_res = evaluate_model(model, [ood_arg_embed, ood_pred_embed, ood_plain_features], ood_y)
print()
print(pd.DataFrame([ev_res], columns = ['keras_accur', 'keras_loss', 'f1_micro', 'f1_macro', 'accur']))

NameError: name 'ind_plain_features' is not defined

In [52]:
model = construct_plain_model_sparse((ind_plain_features.shape[1],))
model.summary()
model_eval = evaluate_out_of_domain(model, 
                                    [ind_arg_embed, ind_pred_embed, ind_plain_features], ind_y, 
                                    [ood_arg_embed, ood_pred_embed, ood_plain_features], ood_y)
print(model_eval)
describe_cv_result(model_eval)

NameError: name 'ind_plain_features' is not defined

### Context

In [53]:
model = construct_graph_lstm_model((ind_plain_features.shape[1],))
model.summary()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
model.fit([
           #ind_arg_context, 
        #ind_pred_context,   
        ind_arg_embed, 
        ind_pred_embed, 
        ind_plain_features,
        ind_pred_context], 
#model.fit([ind_arg_context, ind_pred_context, ind_arg_embed, ind_pred_embed, ind_plain_features], 
           ind_y, nb_epoch=6, batch_size=64, validation_split = 0.1, 
          shuffle=True, callbacks = [early_stopping])

#model.evaluate([ood_arg_context, ood_pred_context, ood_arg_embed, ood_pred_embed, ood_plain_features], ood_y)
model.evaluate([
    #    ood_arg_context, 
    #    ood_pred_context,
        ood_arg_embed, 
        ood_pred_embed,
        ood_plain_features,
        ood_pred_context
    ], ood_y)

NameError: name 'ind_plain_features' is not defined

In [54]:
model = construct_graph_lstm_model((ind_plain_features.shape[1],))
model.summary()
model_eval = evaluate_out_of_domain(model, 
                                    [ind_arg_embed, ind_pred_embed, ind_plain_features, ind_pred_context], ind_y, 
                                    [ood_arg_embed, ood_pred_embed, ood_plain_features, ood_pred_context], ood_y)
print(model_eval)
describe_cv_result(model_eval)

NameError: name 'ind_plain_features' is not defined

# Evaluation

In [55]:
# Use only for model comparision. Do not use for training.

In [96]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score


def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, *args, **kwargs):
    model.fit(X_train, y_train, *args, **kwargs)
    
    keras_eval = model.evaluate(X_test, y_test)
    
    pred = model.predict(X_test).argmax(axis = 1)
    f1_micro = f1_score(pred, y_test.argmax(axis = 1), average = 'micro')
    f1_macro = f1_score(pred, y_test.argmax(axis = 1), average = 'macro')
    accur = accuracy_score(pred, y_test.argmax(axis = 1))
    
    return list(keras_eval) + [f1_micro, f1_macro, accur]
    

def custom_cross_val(cr_f, X, y, cv, *args, **kwargs):
    cr_f().summary()
    eval_res = list()
    for i, (train, test) in enumerate(cv.split(y)):
        model = cr_f()
        print('Running Fold', i+1, '/', cv.n_splits)
        eval1 = train_and_evaluate_model(model, 
                                         [X[j][train] for j in range(len(X))], y[train], 
                                         [X[j][test] for j in range(len(X))], y[test], 
                                         *args, **kwargs)
        
        print()
        print('Fold result: ', eval1)
        eval_res.append(eval1)
    
    return np.array(eval_res)


def describe_cv_result(cv_res):
    print(cv_res)
    mean_cv_res = cv_res.mean(axis = 0)
    std_cv_res = cv_res.std(axis = 0)
    print('Mean')
    print(pd.DataFrame([mean_cv_res], columns = ['loss', 'keras_accur', 'micro_f1', 'macro_f1', 'accur']))
    print('Std')
    print(pd.DataFrame([std_cv_res], columns = ['loss', 'keras_accur', 'micro_f1', 'macro_f1', 'accur']))
    
    
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=42)

## Simple

In [98]:
curr_features = np.concatenate((plain_features, e_verbs, e_args), axis = 1)
cv_res = custom_cross_val(lambda : construct_plain_model((curr_features.shape[1],)), 
                          [curr_features], 
                          y, cv = cv, epochs=13, batch_size=64,
                          validation_split = 0., shuffle=True, verbose = 0)

describe_cv_result(cv_res)

Plain model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 600)               1788600   
_________________________________________________________________
dropout (Dropout)            (None, 600)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 400)               240400    
_________________________________________________________________
batch_normalization (BatchNo (None, 400)               1600      
_________________________________________________________________
activation (Activation)      (None, 400)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 44)                17644   

KeyboardInterrupt: 

In [85]:
cv_res = custom_cross_val(lambda : construct_plain_model((plain_features.shape[1],)), 
                          [plain_features], 
                          y, cv = cv, epochs=13, batch_size=64,
                          validation_split = 0., shuffle=True, verbose = 0)

describe_cv_result(cv_res)

Plain model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 600)               565800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               240400    
_________________________________________________________________
batch_normalization_1 (Batch (None, 400)               1600      
_________________________________________________________________
activation_1 (Activation)    (None, 400)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 34)                13634   

In [86]:
single_chunk = np.concatenate((e_args, e_verbs, plain_features), axis = 1)
cv_res = custom_cross_val(lambda : construct_plain_model((single_chunk.shape[1],)), 
                          [single_chunk], 
                          y, cv = cv, epochs=13, batch_size=64,
                          validation_split = 0., shuffle=True, verbose = 0)

describe_cv_result(cv_res)

Plain model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 600)               925800    
_________________________________________________________________
dropout_13 (Dropout)         (None, 600)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 400)               240400    
_________________________________________________________________
batch_normalization_13 (Batc (None, 400)               1600      
_________________________________________________________________
activation_13 (Activation)   (None, 400)               0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 400)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 34)                13634   

## Sparse

In [89]:
cv_res = custom_cross_val(lambda : construct_plain_model_sparse(plain_features.shape[1], 
                                                                embeddings.vector_size, 
                                                                y.shape[1]), 
                          [embedded_args, embedded_verbs, plain_features], y, 
                          cv = cv, epochs=13, batch_size=300,
                          validation_split = 0., shuffle=True, verbose = 0)

describe_cv_result(cv_res)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
pred_embed (InputLayer)         (None, 300)          0                                            
__________________________________________________________________________________________________
arg_embed (InputLayer)          (None, 300)          0                                            
__________________________________________________________________________________________________
input_categorical (InputLayer)  (None, 990)          0                                            
__________________________________________________________________________________________________
dense_92 (Dense)                (None, 100)          30100       pred_embed[0][0]                 
__________________________________________________________________________________________________
dense_93 (

## Context

In [ ]:
cv_res = custom_cross_val(lambda : construct_graph_lstm_model((plain_features.shape[1],)), 
                          [arg_context_embedded, 
                           pred_context_embedded, 
                           embedded_args, 
                           embedded_verbs,
                           plain_features], y, 
                          cv = cv, epochs=6, batch_size=64, validation_split = 0., 
                          shuffle=True)

describe_cv_result(cv_res)

# Training and predicting

In [ ]:
# Training model and saving

In [87]:
from sklearn.model_selection import train_test_split

train_ids, test_ids = train_test_split(X_orig.ex_id.unique(), test_size=0.2, random_state=42)
train_ids = set(train_ids.tolist())
test_ids = set(test_ids.tolist())

In [88]:
train_selector_pd = X_orig.ex_id.isin(train_ids)
test_selector_pd = X_orig.ex_id.isin(test_ids)
train_selector = train_selector_pd.values
test_selector = test_selector_pd.values

In [89]:
train_data = {k : data[k] for k in train_ids}
test_data = {k : data[k] for k in test_ids}

with open(os.path.join(main_model_path, 'train_data.json'), 'w') as f:
    json.dump(train_data, f)

with open(os.path.join(main_model_path, 'test_data.json'), 'w') as f:
    json.dump(test_data, f)

In [77]:
def select_from_nparray_list(nparray_list, selector):
    return [e[selector] for e in nparray_list]

model = construct_plain_model_sparse(plain_features.shape[1], embeddings.vector_size, y.shape[1])
model.fit(select_from_nparray_list([embedded_args, embedded_verbs, plain_features], train_selector),
          select_from_nparray_list([y], train_selector), 
          epochs=10, batch_size=64, validation_split = 0.1, shuffle=True)

model.evaluate(select_from_nparray_list([embedded_args, embedded_verbs, plain_features], test_selector), 
               select_from_nparray_list([y], test_selector))
model.save(os.path.join(main_model_path, 'neural_model.h5'))

Train on 37893 samples, validate on 4211 samples
Epoch 1/10
37893/37893 [==============================] - 11s 290us/step - loss: 1.9349 - acc: 0.5656 - val_loss: 1.1430 - val_acc: 0.7421
Epoch 2/10
37893/37893 [==============================] - 9s 232us/step - loss: 1.1200 - acc: 0.7305 - val_loss: 0.8598 - val_acc: 0.7727
Epoch 3/10
37893/37893 [==============================] - 9s 232us/step - loss: 0.9209 - acc: 0.7613 - val_loss: 0.7801 - val_acc: 0.7822
Epoch 4/10
37893/37893 [==============================] - 9s 231us/step - loss: 0.8213 - acc: 0.7741 - val_loss: 0.7397 - val_acc: 0.7903
Epoch 5/10
37893/37893 [==============================] - 9s 229us/step - loss: 0.7521 - acc: 0.7903 - val_loss: 0.7198 - val_acc: 0.7948
Epoch 6/10
37893/37893 [==============================] - 9s 230us/step - loss: 0.7003 - acc: 0.7981 - val_loss: 0.7003 - val_acc: 0.7920
Epoch 7/10
37893/37893 [==============================] - 9s 231us/step - loss: 0.6519 - acc: 0.8097 - val_loss: 0.6985 - 

[0.6926666886275118, 0.8026800037746532]

In [81]:
# Hold-out evaluation.

evaluate_model(model,
               select_from_nparray_list([embedded_args, embedded_verbs, plain_features], test_selector), 
               select_from_nparray_list([y], test_selector)[0])

10597/10597 [==============================] - 1s 108us/step


array([0.69266669, 0.80268   , 0.80268   , 0.77108236, 0.80268   ])

In [84]:
pred = model.predict(select_from_nparray_list([embedded_args, embedded_verbs, plain_features], test_selector))

test_examples_to_store = X_orig.loc[test_selector_pd[test_selector_pd].index, :].loc[:, ['arg_address', 'ex_id']]
test_data = {k : data[k] for k in test_ids}


for index, (pd_index, row) in enumerate(test_examples_to_store.iterrows()):
    ex = test_data[row['ex_id']]
    arg_addr = row['arg_address']
    sent = ex[arg_addr[0]]
    token = sent[arg_addr[1]]
    cl = pred[index]
    predicted_role = label_encoder.inverse_transform(np.array([cl]))[0]
    actual_role = label_encoder.inverse_transform(np.array([select_from_nparray_list([y], test_selector)[0][index]]))[0]
    
    token['rolepred1'] = actual_role
    token['rolepred2'] = predicted_role

In [85]:
with open('./test_data_ann_1.json', 'w') as f:
    json.dump(test_data, f)

# Brat convertion

In [ ]:
# Converts results to brat annotation for inspecting.
# Needs framebank_preprocessing from http://nlp.isa.ru/framebank_parser/

In [ ]:
!python2.7 ./framebank_preprocessing/convert_corpus_to_brat.py --inputFile=./test_data_ann_1.json --outputDir=./brat_ann2/

In [ ]:
!python2.7 ./framebank_preprocessing/convert_corpus_to_brat.py --inputFile=./test_data_ann_1.json --outputDir=./syntaxnet_1/ --converter=syn

In [54]:
!export PYTHONPATH=../ && python2.7 ./convert_corpus_to_brat.py --inputFile=./test_data_ann_1.json --outputDir=./brat_ann2/

2018-03-11 20:56:20 - Loading corpus data...
2018-03-11 20:56:22 - Done.
2018-03-11 20:56:22 - Creating verb-example index...
2018-03-11 20:56:22 - Done.
2018-03-11 20:56:22 - Converting and saving...
2018-03-11 20:56:23 - Done.
2018-03-11 20:56:23 - Generating brat configuration files...
2018-03-11 20:56:23 - Done.
